In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import median
import re
import sys

data = pd.read_csv('TAG Data.csv')

## Reading in Ratio Median

In [132]:
data = pd.read_csv('TAG Data.csv')

data = data[['gene' , 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
data ['Ratios: HLG / ALL (Whole)'] = data ['R5-S#11-HLG_WHOLE_normalized_reads'] / data['R5-S#11-ALL_WHOLE_normalized_reads']

data = data.sort_values('gene')
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()

data = data.drop(['R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads'], axis = 1)

data = pd.DataFrame(data)

mean = data.groupby(['gene']).mean()
mean.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Ratio Mean'}, inplace = True)
# mean

median = data.groupby(['gene']).median()
median.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Ratio Median'}, inplace = True)
# median

count = data.groupby(['gene']).count()
count.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Count'}, inplace = True)
# count

result = data.reset_index(drop = True)
result = pd.concat([mean, median, count], axis = 1)

result = result.sort_values(by = ['Ratio Median'], ascending = False)

greater = result[result['Count'] < 3].index
result.drop(greater, inplace = True)

result.to_csv("GroupBy.csv", header = True)

result = pd.read_csv("GroupBy.csv")
result = result.rename(columns = {'gene' : "Gene"})

result.to_csv("GroupBy.csv", header = True)

# result

## Cre number to GO term

In [4]:
dffile =  'C:\Users\jazzy\Desktop\School\Jinkerson Lab\Data\May 13, 2020\Creinhardtii_281_v5.5.annotation_info.txt'
header = ['number','cre','cre_transcript','cre_transcript2','pfam','pthr',
          'kog','ec','kass','go','ath','ath_name','ath_desc']
df_annotations = pd.read_csv(dffile, skipinitialspace = True, sep = '\t', 
                             header = None, names = header)

# make dict from cre to go
d_cre_to_go = dict(zip(df_annotations['cre'],df_annotations['go']))

# convert entries into lists
for gene in d_cre_to_go.iterkeys():
    if isinstance(d_cre_to_go[gene], (str, unicode)): # is it a string (and not np.nan?)
        d_cre_to_go[gene] = d_cre_to_go[gene].split(',')
        
d_cre_to_go = pd.DataFrame.from_dict(d_cre_to_go, orient = 'index')

d_cre_to_go.to_csv("GOTerms.csv", header = True)

d_cre_to_go = pd.read_csv("GOTerms.csv")

d_cre_to_go = d_cre_to_go.rename(columns = {"Unnamed: 0" : "Gene", "0" : "GO Term"})

d_cre_to_go.to_csv("GOTerms.csv", header = True)

#d_cre_to_go

## Extracted gene, confidence level, feature, and IB from large-lib_full_2016.10.25
## Only keep the genes with a max confidence level of 4
## And reorder the confidence level from least to greatest
## Remove ".v5.5"

In [130]:
data = pd.read_csv('large-lib_full_2016.10.25.csv')

data = data.drop(['well', 'side', 'chromosome', 'plate', 'strand', 
                  'min_position', 'full_position', 'orientation', 
                  'gene_end_distances', 'total_reads', 
                  'if_both_sides', 'LEAPseq_distance', 
                  'LEAPseq_percent_confirming',
                  'LEAPseq_N_confirming', 'LEAPseq_N_nonconf', 
                  'LEAPseq_N_unique_pos',
                  'flanking_seq', 'plate_errors', 'well_errors', 
                  'plate_rpm', 'well_rpm',
                  'if_fixed_position'], axis = 1)

filterData = data[data['confidence_level'] <= 4]
filterData = filterData.sort_values('confidence_level')
filterData = filterData.rename(columns = {"gene" : "Gene",
                                          "confidence_level" : "Confidence Level",
                                          "feature" : "Feature"})
filterData = filterData.reset_index(drop = True)

genie = filterData["Gene"].astype(str).str.replace(r'.v5.5$', '')
genie = genie.str.replace(r".v5.5", '').str.strip()

filterData = filterData.drop(columns = {"Gene"})
filterData["Gene"] = genie

filterData =  filterData[["Gene", "Confidence Level", "IB", "Feature"]]
# filterData ["Feature"] = filterData["Feature"].str.capitalize()
filterData.to_csv("ConfidenceLevels.csv", header = True)

# filterData

## Big Merge

In [176]:
df1 = pd.read_csv('GroupBy.csv', sep = ',', dtype = str)
df2 = pd.read_csv('GOTerms.csv', sep = ',', dtype = str)
df3 = pd.read_csv('ConfidenceLevels.csv', sep = ',', dtype = str)

# bigMerge = df1.merge(df2, on = 'Gene').merge(df3, on = 'Gene')

# bigMerge = bigMerge.drop(columns = ["Unnamed: 0_x", "Unnamed: 0_y", 
#                                     "Unnamed: 0"])
# # bigMerge = df1.merge(df2, how = 'outer', on = "Gene", validate = "m:m")
# # # bigMerge = bigMerge.drop(columns = "Unnamed: 0")

# bigMerge = bigMerge[["Gene", "Ratio Median", "Ratio Mean", "GO Term", 
#                      "Feature", "Confidence Level", "IB"]]

# # bigMerge.to_csv("BigMerge.csv", header = True)

# bigMerge

leftBigMerge = pd.merge(left = df1, right = df2, how = 'left', 
                        left_on = 'Gene', right_on = 'Gene')
leftBigMerge = pd.merge(left = leftBigMerge, right = df3, how = 'left',
                        left_on = 'Gene', right_on = 'Gene')
leftBigMerge = leftBigMerge.drop(columns = ["Unnamed: 0_x", "Unnamed: 0_y", 
                                    "Unnamed: 0"])
leftBigMerge = leftBigMerge[["Gene", "Ratio Median", "Ratio Mean", "GO Term", 
                             "Feature", "Confidence Level", "IB"]]

# feat = leftBigMerge["Feature"].copy()
# leftBigMerge["Feature"] = leftBigMerge["Feature"].str.cat(feat, sep = ", ")
leftBigMerge["Feature"] = leftBigMerge["Feature"].astype(str)

# level = leftBigMerge["Confidence Level"].copy()
# level = level.astype(str)
leftBigMerge["Confidence Level"] = leftBigMerge["Confidence Level"].astype(str)
# leftBigMerge["Confidence Level"] = leftBigMerge["Confidence Level"].str.cat(level, sep = ", ")

# bar = leftBigMerge["IB"].copy()
# leftBigMerge["IB"] = leftBigMerge["IB"].str.cat(bar, sep = ", ")
leftBigMerge["IB"] = leftBigMerge["IB"].astype(str)

# leftBigMerge = leftBigMerge.drop_duplicates()

# leftBigMerge = leftBigMerge[["Gene", "Ratio Median", "Ratio Mean", 
#                                         "GO Term", "Feature", "Confidence Level",
#                                         "IB"]].groupby(["Gene", "Ratio Median", 
#                                                         "Ratio Mean", "GO Term"])["Feature", 
#                                                                                   "Confidence Level", 
#                                                                                   "IB"].transform(lambda x: ", ".join(x))
leftBigMerge.head()

100000000 loops, best of 3: 19.6 ns per loop


## Keep rows that have GO Terms and the first of the repeated Gene

In [114]:
slim = bigMerge[pd.notnull(bigMerge["GO Term"])]

slim = slim.drop_duplicates(subset = 'Gene', keep = "first")
slim = slim.reset_index(drop = True)

slim.to_csv("SlimShady.csv", header = True)

## Definition

In [148]:
defo = pd.read_csv('GO to Description.csv')

defo = defo.drop(columns = ["holds_over_chain", "Unnamed: 0", "is_a", 
                            "is_class_level", "is_metadata_tag", "is_obsolete",
                            "is_transitive", "name", "namespace", "relationship",
                            "replaced_by", "subset", "synonym", "transitive_over",
                            "xref"])
defo = defo[["id", "alt_id", "consider", "def", "comment"]]
defo = defo.rename(columns = {"id" : "ID", "alt_id" : "Alternate ID",
                              "consider" : "Consider", "def" : "Definition",
                              "comment" : "Comment"})

defo.to_csv("Definitions Maybe.csv", header = True) 

# defo

## Info from the JGI file
## Remove ".t.1.1"

In [147]:
desc = pd.read_csv('Creinhardtii_281_v5.6.description.JGI.csv')

desc = desc.rename(columns = {"Column1" : "Gene", "Column2" : "Description"})
genie = filterData["Gene"].astype(str).str.replace(r'.t1.1$', '')

desc = desc.drop(columns = {"Gene"})
desc["Gene"] = genie
desc = desc[["Gene", "Description"]]

desc.to_csv("Descriptions.csv", header = True) 

## Based of of the gene name, with description

In [193]:
data1 = pd.read_csv("BigMerge.csv", sep = ',', dtype = str)
data2 = pd.read_csv("Descriptions.csv", sep = ',')

superLeft = pd.merge(left = data1, right = data2, how = 'left', 
                     left_on = 'Gene', right_on = 'Gene')
superLeft = superLeft.drop(columns = {"Unnamed: 0_x", "Unnamed: 0_y"})

superLeft.to_csv("Mega Large Merge.csv", header = True) 

superLeft = superLeft.set_index('IB')

superLeft.to_csv("Mega Large Merge.csv", header = True) 

superLeft.head()


,Gene,Ratio Median,Ratio Mean,GO Term,Feature,Confidence Level,Description
IB,,,,,,,
GTTTGGACGCAATAATTTATGT,Cre08.g369667,200.06134301270416,135.72555963803572,"['GO:0004672', 'GO:0005524', 'GO:0006468']",3'UTR,1,NaN
TGGCGTTAGGTCAGCTGGAGGC,Cre08.g369667,200.06134301270416,135.72555963803572,"['GO:0004672', 'GO:0005524', 'GO:0006468']",3'UTR,1,NaN
TGCCCGGCCACGGTGAGCCACA,Cre08.g369667,200.06134301270416,135.72555963803572,"['GO:0004672', 'GO:0005524', 'GO:0006468']",3'UTR,3,NaN
TCGGCCAGGGCTACCAGTAAGA,Cre08.g369667,200.06134301270416,135.72555963803572,"['GO:0004672', 'GO:0005524', 'GO:0006468']",intron,3,NaN
ACTCCTTGGTGGAGATCGCAGA,Cre08.g369667,200.06134301270416,135.72555963803572,"['GO:0004672', 'GO:0005524', 'GO:0006468']",intron,4,NaN
